## Imports

In [ ]:
%pip install -q datasets transformers==4.28.0 --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 9.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Constants

In [ ]:
model_id = "gpt2"
DEVICE = 'cuda'

## Load Dataset

In [ ]:
from datasets import load_dataset

# Note: this is the same dataset as https://urldefense.com/v3/__https://pytorch.org/text/stable/datasets.html*id22__;Iw!!LIr3w8kk_Xxm!oJNtg0Dcg0AZd7jpP-TKv-pOUtoxBQ668RwcOjO1YIHTzTC8ZBVbXkyntoc9YijqdBKbGukpcgzchLbesQ$
raw_dataset = load_dataset("wikitext", "wikitext-2-v1")

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

## Load Tokenizer

In [ ]:
from transformers import GPT2TokenizerFast
from transformers import DataCollatorForLanguageModeling


tokenizer = GPT2TokenizerFast.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

## Load Untrained Model

In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=128,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

GPT-2 size: 124.4M parameters


## Data Preprocessing

In [ ]:
processed_dataset = (raw_dataset
    .filter(lambda example: len(example['text']) > 0)
    .map(
        lambda example: tokenizer(
            example['text'], 
            max_length=128, 
            truncation=True, 
            padding='max_length',
            return_tensors="pt",
            return_attention_mask=True,
        ),
        batched=True
    )
)

processed_dataset = processed_dataset.remove_columns('text')
processed_dataset.set_format('torch')

Filter:   0%|          | 0/4358 [00:00<?, ? examples/s]

Filter:   0%|          | 0/36718 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3760 [00:00<?, ? examples/s]

Map:   0%|          | 0/2891 [00:00<?, ? examples/s]

Map:   0%|          | 0/23767 [00:00<?, ? examples/s]

Map:   0%|          | 0/2461 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import DataLoader
import torch

model = model.to(DEVICE)
model_parameters = list(model.parameters())

real_train_dataloader = DataLoader(processed_dataset['train'], batch_size=16)
batch_real = next(iter(real_train_dataloader))

x_real = batch_real['input_ids'].to(DEVICE)
attn_mask_real = batch_real['attention_mask'].to(DEVICE)
y_real = x_real.clone()

out_real = model(x_real, attention_mask=attn_mask_real, labels=y_real)
gradient_weights_real = torch.autograd.grad(out_real.loss, model_parameters)



### Synthetic Data

In [ ]:
# Generate
syn_tokens = torch.randint(0, 50257-1, (120, 128))

In [ ]:
syn_train_dataloader = DataLoader(syn_tokens, batch_size=16)
batch_syn = next(iter(syn_train_dataloader))

# Compute the synthetic loss and get synthetic gradient weights
x_syn = batch_syn.to(DEVICE)
y_syn = x_syn.clone()

# syn_embed = model.get_input_embeddings()(x_syn) # Grab the synthetic embeddings, this is the vector space that we wish to update
# syn_embed.requires_grad = True # It requires grads now
out_syn = model(x_syn, labels=y_syn) # We pass it through the remaining model

In [ ]:
gradient_weights_syn = torch.autograd.grad(out_syn.loss, model_parameters, create_graph=True) # We obtain embeddings for all layers, including embeddings layer.

In [ ]:
syn_train_dataloader = DataLoader(syn_tokens, batch_size=16)
batch_syn = next(iter(syn_train_dataloader))

# Compute the synthetic loss and get synthetic gradient weights
x_syn = batch_syn.to(DEVICE)
y_syn = x_syn.clone()

syn_embed = model.get_input_embeddings()(x_syn) # Grab the synthetic embeddings, this is the vector space that we wish to update
# syn_embed.requires_grad = True # It requires grads now
syn_embed = syn_embed.detach()
syn_embed.requires_grad = True
out_syn = model(inputs_embeds=syn_embed, labels=y_syn) # We pass it through the remaining model

In [ ]:
(out_syn.loss)

tensor(10.9905, device='cuda:0', grad_fn=<NllLossBackward0>)